[Initial Environment]

In [1]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import sys
import os
sys.path.append(os.path.join(".."))
import libPython.core.util

In [1]:
import logging
logging.basicConfig( 
        level=logging.DEBUG,
        # level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(module)s.%(funcName)s():%(lineno)d %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
        )
logging.info("this is message")

2021-12-16 20:44:40 [INFO] <ipython-input-1-f5c87b5c617d>.<module>():8 this is message


In [4]:
import logging

logging.basicConfig( 
        level=logging.DEBUG,
        # level=logging.INFO,
        format=None,
        datefmt=None
        )
logging.info("this is message")

this is message


In [15]:
l = [ 9,9,1,2,4,4,5]
print(l)
s=set(l)
print(s)
l2=list(s)
print(l2[-1])

[9, 9, 1, 2, 4, 4, 5]
{1, 2, 4, 5, 9}
9


In [2]:
from FinMind.data import DataLoader

api = DataLoader()
# api.login_by_token(api_token='token')
# api.login(user_id='user_id',password='password')
df = api.taiwan_daily_short_sale_balances(
    stock_id="2330",
    start_date='2020-04-01',
    end_date='2020-04-12'
)

In [7]:
from finlab import data
data.get('price:收盤價')

ModuleNotFoundError: No module named 'finlab'

In [25]:
import dateutil.parser
import re

# line = r"[2022-03-12 19:00:37.955] GTM: temp = 24106, cur freq = 214000 KHz, ttj = 95000, bless=1.1 god-12.22"
# line = r"[2022-03-12 19:00:37.955] temp = 24106, cur freq = 214000 KHz, ttj = 95000, bless=1.1 god=12.22"
line = r" <idle>-0 [005] 387.386832: sched_wakeup: comm=met-cmd pid=6840 prio=120 success=1 target_cpu=005 state=R"


def parse_keyvalue_pairs(line):
    ret_keyvalue = []
    # key_value_list = re.findall(r"\w+\s*=\s*[^, ]+",line)
    key_value_list = re.findall(r"[^,=:\[\]]+\s*=\s*[^, ]+", line)
    for pair in key_value_list:
        idx = pair.find("=")
        key = pair[:idx].strip()
        value = pair[idx+1:].strip()
        ret_keyvalue.append((key, value))
    return ret_keyvalue

def parse_ftrace_keyvalue_pairs(line):
    ret_ts = None
    ret_keyval = []
    m = re.match(r".*\[\d+\]\s*([\d.]+):\s*(\S*):\s*(.*)", line)
    if not m:
        return None
    ret_ts = float(m.group(1))
    trace_name = m.group(2)
    key_value_str = m.group(3)
    ret_keyval = parse_keyvalue_pairs(key_value_str)
    ret_keyval = [ (trace_name + ":" + p[0], p[1]) for p in ret_keyval] 
    return (ret_ts, ret_keyval)

def parse_teraterm_keyvalue_pairs(line):
    ret_ts = None
    ret_keyval = []
    m = re.match(r"\s*\[(.+)\]\s*(([\S^:]+):)?(.*)", line)
    if not m:
        return None
    ret_ts = dateutil.parser.parse (m.group(1), fuzzy=True).timestamp()
    trace_name = m.group(3)
    key_value_str = m.group(4)
    ret_keyval = parse_keyvalue_pairs(key_value_str)
    if trace_name:
        ret_keyval = [ (trace_name + ":" + p[0], p[1]) for p in ret_keyval]
    return (ret_ts, ret_keyval)

def ParsekeyValue(line):
    line = line.strip()
    if line.startswith("["):
        return parse_teraterm_keyvalue_pairs(line)
    else:
        return parse_ftrace_keyvalue_pairs(line)


# print(parse_ftrace_keyvalue_pairs(line))
print(ParsekeyValue(line))

(387.386832, [('sched_wakeup:comm', 'met-cmd'), ('sched_wakeup:pid', '6840'), ('sched_wakeup:prio', '120'), ('sched_wakeup:success', '1'), ('sched_wakeup:target_cpu', '005'), ('sched_wakeup:state', 'R')])


In [3]:

import libPython.core.generic_parser as generic_parser
# line = r"[2022-03-12 19:00:37.955] GTM: temp = 24106, cur freq = 214000 KHz, ttj = 95000, bless=1.1 god-12.22"
# line = r"[2022-03-12 19:00:37.955] temp = 24106, cur freq = 214000 KHz, ttj = 95000, bless=1.1 god=12.22"
line = r" <idle>-0 [005] 387.386832: sched wakeup: comm=met-cmd pid=6840 prio=120 success=1 target_cpu=005 state=R"
# print(parse_ftrace_keyvalue_pairs(line))
# print(parse_teraterm_keyvalue_pairs(line))
print(generic_parser.Parsekeyvalue(line))

ModuleNotFoundError: No module named 'libPython.core.generic_parser'